In [1]:
import pandas as pd
import requests
import datetime
import time
import os
import sys
from dateutil.rrule import rrule, MONTHLY
from dateutil.relativedelta import relativedelta

In [2]:
def Atomic(limit, time_end, time_start,lines_to_save_data):
    print('SET:', time_start)
    counter = 0
    df = pd.DataFrame({}, dtype=str)
    url = '''https://api.etherscan.io/api
   ?module=transaction
   &action=gettxreceiptstatus
   &txhash=0xbc8412594ffa7382e8306215ece5b4e041fc641b18133221925a4ac6ecf15432
   &apikey=W18VDDEC8JNGSR5B7F5FP2ANA4X33NUWWK'''
    
    state = {'sold':'3'}
    page=1
    
    time_start = int(time_start.timestamp()*10**3)
    time_end = int(time_end.timestamp()*10**3)

    while(time_end>time_start):
        # print('START REQUEST')
        # print(time_start)

        querystring = {"state":state["sold"],
              "after":time_start,
              "page":str(page),
              "limit":str(limit)}
        response = requests.request("GET", url, params=querystring)

        if response.status_code==200:
            
            
            df_supp = pd.DataFrame(response.json()['data'])
            print(len(df_supp))
            df_supp['created_at_time'] = pd.to_numeric(df_supp['created_at_time'])

            df_supp = df_supp[df_supp['created_at_time'] < time_end]
            
            print(len(df_supp))

            if len(df_supp)==0 :  
                break
            
            # df = df.append(df_supp, ignore_index=True)
            # time_data_supp = int(df_supp.created_at_time.max())
            # if time_data_supp > time_start : 
            #     time_start = time_data_supp
            # page += 1
            # print(len(df_supp), len(df), pd.to_datetime(time_end, unit='ms'), pd.to_datetime(time_data_supp, unit='ms'), page)
        else:
            print('REQUEST FAILED')
            return
        
        counter +=limit

In [3]:
def downloadData(dates=pd.to_datetime({'year': [2021, 2021],'month': [1,10],'day': [1,1]}), download_function=Atomic):
    print('From: ', dates[0])
    print('To: ', dates[1])
    limit = 100
    for curr_month in rrule(freq=MONTHLY, dtstart=dates[0], until=dates[1]):
        nextMonth = curr_month + relativedelta(months=1)
        download_function(limit,nextMonth,curr_month,5000)

downloadData()

From:  2021-01-01 00:00:00
To:  2021-10-01 00:00:00
SET: 2021-01-01 00:00:00
100
0
SET: 2021-02-01 00:00:00
100
0
SET: 2021-03-01 00:00:00
100
0
SET: 2021-04-01 00:00:00


KeyboardInterrupt: 